In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from uszipcode import SearchEngine

c:\Users\markm\AppData\Local\Programs\Python\Python310\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Import Data
Get data from different sources before combining
* Cleaned up EV data: TX_WA_CO_NY.csv
* Average EV price data over time: Avg_EV_Price.csv

In [6]:
df = pd.read_csv('.\Data\TX_WA_CO_NY.csv')
df_ev = pd.read_csv('.\Data\Avg_EV_Price.csv')
df

,State,ZIP Code,Registration Date,Drivetrain Type,Vehicle Count
0,TX,75001,2019-11-01,PHEV,2
1,TX,75001,2020-01-01,PHEV,9
2,TX,75001,2020-02-01,BEV,4
3,TX,75001,2020-04-01,BEV,6
4,TX,75001,2020-05-01,PHEV,16
...,...,...,...,...,...
229929,NY,14905,2022-12-01,PHEV,6
229930,NY,14905,2023-01-01,BEV,5
229931,NY,14905,2023-01-01,PHEV,6
229932,NY,14905,2023-02-01,BEV,1


In [11]:
df_ev['Month'] = pd.to_datetime(df_ev['Month'], format='%b-%y')
df_ev['Month']

0    2020-01-01
1    2020-02-01
2    2020-03-01
3    2020-04-01
4    2020-05-01
5    2020-06-01
6    2020-07-01
7    2020-08-01
8    2020-09-01
9    2020-10-01
10   2020-11-01
11   2020-12-01
12   2021-01-01
13   2021-02-01
14   2021-03-01
15   2021-04-01
16   2021-05-01
17   2021-06-01
18   2021-07-01
19   2021-08-01
20   2021-09-01
21   2021-10-01
22   2021-11-01
23   2021-12-01
24   2022-01-01
25   2022-02-01
26   2022-03-01
27   2022-04-01
28   2022-05-01
29   2022-06-01
30   2022-07-01
31   2022-08-01
32   2022-09-01
33   2022-10-01
34   2022-11-01
35   2022-12-01
36   2023-01-01
37   2023-02-01
Name: Month, dtype: datetime64[ns]